In [1]:
from neo4j import GraphDatabase

URI = "bolt://localhost:7687"
AUTH = ('neo4j', 'neo4j1234')

In [71]:
def findATrip(srcCity, dstCity):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match p=(s:Airport)-->*(d: Airport) where s.city = $src and d.city = $dst return relationships(p), nodes(p) limit 1'
        records, _, _ = driver.execute_query(query, src=srcCity, dst=dstCity)
        driver.close()
        
    #Extract
    trips = []
    for record in records:
        routes = record.get('relationships(p)')
        nodes = record.get('nodes(p)')
        
        path = []
        for i in range(len(routes)):
            src, dst, route = nodes[i], nodes[i + 1], routes[i]
            airline = route.get("airline_name")
            dept = src.get("IATA") + ", " + src.get("city")
            dest = dst.get("IATA") + ", " + dst.get("city")
            path.append(f"{airline} airline: {dept}  to  {dest}")
        
        trips.append(path)
    
    for t in trips:
        print('Trip: ', t)
    return trips


In [72]:
findATrip("Seattle", "Las Vegas")

Trip:  ['WN airline: SEA, Seattle  to  LAS, Las Vegas']


[['WN airline: SEA, Seattle  to  LAS, Las Vegas']]

In [2]:
def findATripWithDHops(srcCity, dstCity, dHops):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = "match p=(s:Airport)-->{1," + str(dHops) + "}(d: Airport) where s.city = $src and d.city = $dst return relationships(p), nodes(p) limit 1"
        records, _, _ = driver.execute_query(query, src=srcCity, dst=dstCity)
        driver.close()
        
    #Extract
    trips = []
    for record in records:
        routes = record.get('relationships(p)')
        nodes = record.get('nodes(p)')
        
        path = []
        for i in range(len(routes)):
            src, dst, route = nodes[i], nodes[i + 1], routes[i]
            airline = route.get("airline_name")
            dept = src.get("IATA") + ", " + src.get("city")
            dest = dst.get("IATA") + ", " + dst.get("city")
            path.append(f"{airline} airline: {dept}  to  {dest}")
        
        trips.append(path)
    
    for t in trips:
        print('Trip: ', t)
    return trips


In [3]:
findATripWithDHops("Seattle", "Las Vegas", 10)

Trip:  ['WN airline: SEA, Seattle  to  LAS, Las Vegas']


[['WN airline: SEA, Seattle  to  LAS, Las Vegas']]

In [4]:
def findCitiesWithDHops(srcCity, dHops):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = "match p=(s:Airport)-->{1," + str(dHops) + "}(dst: Airport) where s.city = $src return distinct dst.city order by dst.city"
        records, _, _ = driver.execute_query(query, src=srcCity)
        cities = list(map(lambda r: r.get('dst.city'), records))
        driver.close()
    
    print(f"{len(cities)} cities found: ")
    print(cities)
    return cities

In [5]:
findCitiesWithDHops("Seattle", 2)

415 cities found: 
['Aberdeen', 'Abilene', 'Akron', 'Alamosa', 'Albany', 'Albuquerque', 'Alexandria', 'Allentown', 'Alliance', 'Alpena', 'Altoona', 'Amarillo', 'Anchorage', 'Aniak', 'Appleton', 'Arcata CA', 'Asheville', 'Aspen', 'Atlanta', 'Atlantic City', 'Augusta', 'Austin', 'BRISTOL', 'Bakersfield', 'Baltimore', 'Bangor', 'Bar Harbor', 'Barnstable', 'Baton Rouge', 'Beaumont', 'Beckley', 'Bella Coola', 'Bellingham', 'Bemidji', 'Bentonville', 'Bethel', 'Billings', 'Binghamton', 'Birmingham', 'Bismarck', 'Bloomington', 'Boise', 'Boston', 'Bozeman', 'Brainerd', 'Branson', 'Brownsville', 'Brunswick', 'Buffalo', 'Burbank', 'Burlington', 'Bush Field', 'Butte', 'Calgary', 'Campbell River', 'Cape Girardeau', 'Carlsbad', 'Casper', 'Castlegar', 'Cedar City', 'Cedar Rapids', 'Champaign', 'Charleston', 'Charlotte', 'Charlottesville VA', 'Charlottetown', 'Chattanooga', 'Cheyenne', 'Chicago', 'Chico', 'Cincinnati', 'Cleveland', 'Cody', 'Cold Bay', 'College Station', 'Colorado Springs', 'Columbia',

['Aberdeen',
 'Abilene',
 'Akron',
 'Alamosa',
 'Albany',
 'Albuquerque',
 'Alexandria',
 'Allentown',
 'Alliance',
 'Alpena',
 'Altoona',
 'Amarillo',
 'Anchorage',
 'Aniak',
 'Appleton',
 'Arcata CA',
 'Asheville',
 'Aspen',
 'Atlanta',
 'Atlantic City',
 'Augusta',
 'Austin',
 'BRISTOL',
 'Bakersfield',
 'Baltimore',
 'Bangor',
 'Bar Harbor',
 'Barnstable',
 'Baton Rouge',
 'Beaumont',
 'Beckley',
 'Bella Coola',
 'Bellingham',
 'Bemidji',
 'Bentonville',
 'Bethel',
 'Billings',
 'Binghamton',
 'Birmingham',
 'Bismarck',
 'Bloomington',
 'Boise',
 'Boston',
 'Bozeman',
 'Brainerd',
 'Branson',
 'Brownsville',
 'Brunswick',
 'Buffalo',
 'Burbank',
 'Burlington',
 'Bush Field',
 'Butte',
 'Calgary',
 'Campbell River',
 'Cape Girardeau',
 'Carlsbad',
 'Casper',
 'Castlegar',
 'Cedar City',
 'Cedar Rapids',
 'Champaign',
 'Charleston',
 'Charlotte',
 'Charlottesville VA',
 'Charlottetown',
 'Chattanooga',
 'Cheyenne',
 'Chicago',
 'Chico',
 'Cincinnati',
 'Cleveland',
 'Cody',
 'Cold Ba

In [6]:
def findAdjacentAirportNodesOfCity(city):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match (s:Airport)-->(d: Airport) where s.city = $city return d'
        records, _, _ = driver.execute_query(query, city = city)
        driver.close()
    return list(map(lambda r: r.get('d'), records))
        

In [7]:
def findAdjacentAirportNodes(airportNode):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match (s:Airport)-->(d: Airport) where s.airport_id = $airport_id return d'
        records, _, _ = driver.execute_query(query, airport_id = airportNode.get("airport_id"))
        driver.close()
    return list(map(lambda r: r.get('d'), records))
        

In [8]:
def findAirportNodeOfCity(city, airportNode):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match p=(s:Airport)-->(d: Airport) where s.city = $city and d.airport_id = $airport_id return s'
        records, _, _ = driver.execute_query(query, city = city, airport_id = airportNode.get("airport_id"))
        driver.close()
    return records[0].get('s')
        

In [9]:
def findRouteBetweenAirports(src, dst):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match p=(s:Airport)-->(d: Airport) where s.airport_id = $src and d.airport_id = $dst return relationships(p)'
        records, _, _ = driver.execute_query(query, src = src.get("airport_id"), dst = dst.get("airport_id"))
        driver.close()
    return records[0].get('relationships(p)')[0]
        

In [10]:
def bfs(src, dst):
    queue = findAdjacentAirportNodesOfCity(src)
    firstChecks = [d for d in queue if d.get("city") == dst]
    if len(firstChecks) > 0:
        lastNode = {"cur": firstChecks[0], "prev": None}
    else:
        queue = list(map(lambda airport: {"cur": airport, "prev": None}, queue))
        visited = set()
        
        lastNode = None
        while len(queue) > 0:
            top = queue.pop(0)
            curNode = top.get("cur")
            if curNode.get("airport_id") not in visited:
                if curNode.get("city") == dst:
                    lastNode = top
                    break
                
                visited.add(curNode.get("airport_id"))
                for adj in findAdjacentAirportNodes(curNode):
                    if curNode.get("city") == src:
                        continue
                    if adj.get("airport_id") not in visited:
                        queue.append({"cur": adj, "prev": top})
        
    nodes = []
    while lastNode is not None:
        nodes.insert(0, lastNode.get("cur"))
        lastNode = lastNode.get("prev")
    
    if len(nodes) > 0:
        nodes.insert(0, findAirportNodeOfCity(src, nodes[0]))
    # print(nodes)
    
    path = []
    for i in range(len(nodes) - 1):
        src, dst = nodes[i], nodes[i + 1]
        route = findRouteBetweenAirports(src, dst)
        airline = route.get("airline_name")
        dept = src.get("IATA") + ", " + src.get("city")
        dest = dst.get("IATA") + ", " + dst.get("city")
        path.append(f"{airline} airline: {dept}  to  {dest}")
    return path
                    

In [11]:
src = "Seattle"
dst = "Masset"    # Cranbrook

In [12]:
bfs(src, dst)

['AC airline: SEA, Seattle  to  YVR, Vancouver',
 '8P airline: YVR, Vancouver  to  ZMT, Masset']